In [ ]:
from biogeme.database import Database
import pandas as pd
swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
database = Database('swissmetro', swissmetro)

In [ ]:
database.variables

In [90]:
import aesara
import aesara.tensor as aet
import numpy as np
from biogeme.expressions import Beta
from biogeme.database import Database
from aesara.tensor.var import *

class DatabaseShared(Database):
	def __init__(self, name, pandasDatabase):
		super().__init__(name, pandasDatabase)

		for v in self.variables:
			self.variables[v].x = aet.matrix(self.variables[v].name)

class BetaShared(Beta):
	def __init__(self, name, value, lowerbound, upperbound, status):
		super().__init__(name, value, lowerbound, upperbound, status)

		self.sharedVar = aesara.shared(np.array([value]))

	def __add__(self, other):
		if type(other) in [TensorVariable]:
			return self.sharedVar + other
		return super().__add__(other)

	def __sub__(self, other):
		return super().__sub__(other)
	
	def __mul__(self, other):
		if type(other) in [TensorVariable]:
			return self.sharedVar * other
		return super().__mul__(other)

	def __truediv__(self, other):
		if type(other) in [TensorVariable]:
			return self.sharedVar / other
		return super().__truediv__(other)

	def __pow__(self, other):
		return super().__pow__(other)
	
	def __radd__(self, other):
		return super().__radd__(other)

	def __rsub__(self, other):
		return super().__rsub__(other)

	def __rmul__(self, other):
		return super().__rmul__(other)

b_cost = BetaShared("b_cost", 0.2, None, None, 0)
b_time = BetaShared("b_time", 0.1, None, None, 0)

print(b_cost)
print(b_cost + 1)

db = DatabaseShared('swissmetro', swissmetro)
globals().update(db.variables)
print(type(TRAIN_CO.x))
# print(database.data[TRAIN_CO.name])

U_1 = b_cost * TRAIN_CO.x
U_2 = b_cost * CAR_CO.x 
U_3 = b_cost * SM_CO.x 
print('U_1', type(U_1))

print(database.data[[TRAIN_CO.name]].shape)
print(b_cost.sharedVar.eval().shape)
Uvec = aet.concatenate([U_1, U_2, U_3], axis=1)
prob = aet.nnet.softmax(Uvec)

f = aesara.function([db.variables[var].x for var in db.variables], prob, on_unused_input='ignore')

output = f(*(database.data[[var]] for var in db.variables))

output.shape

b_cost(0.2)
(b_cost(0.2) + `1`)
<class 'aesara.tensor.var.TensorVariable'>
U_1 <class 'aesara.tensor.var.TensorVariable'>
(10728, 1)
(1,)


(10728, 3)